In [8]:
import pandas as pd
import numpy as np 

Combine Dragon and Dye PCE Data

In [9]:
dragonDescriptors = pd.read_csv('dragonReduced.csv', index_col=0)

cols = list(dragonDescriptors.columns)

dragonDescriptors.reset_index(drop=True, inplace=True)

In [10]:
qikPharmDyeData = pd.read_csv('QikPropAnd3PointPharmFPDyeData.csv')

qikPharmDyeData.reset_index(drop=True, inplace=True)


In [11]:
dragonDatset = pd.concat([dragonDescriptors[cols[:]], qikPharmDyeData['PCE'], qikPharmDyeData['Molecule keywords']], axis=1)

dragonDatset


,NAME,MW,AMW,Mp,nTA,RBF,nDB,nTB,nAB,nN,...,s1_relPathLength_2,s2_numSharedNeighbors,s3_numSharedNeighbors,s2_numRotBonds,s3_numRotBonds,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,PCE,Molecule keywords
0,10.1016/j.dyepig.2012.02.011,907.17,8.807476,0.747356,8,0.155963,6,4,34,6,...,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0.000000,5.19,phenothiazine
1,10.1039/c0ee00218f,987.28,8.092459,0.717684,9,0.130769,8,1,40,4,...,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0.000000,6.00,"coumarin, triphenylamine"
2,10.1039/c0ee00218f,905.15,7.870870,0.705637,9,0.131148,8,1,35,4,...,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0.000000,6.20,"coumarin, triphenylamine"
3,10.1039/c0ee00218f,823.02,7.620556,0.692030,9,0.131579,8,1,30,4,...,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0.000000,5.50,"coumarin, triphenylamine"
4,10.1016/j.jphotochem.2014.06.001,782.04,9.537073,0.738429,11,0.137931,11,0,11,3,...,0.0,0.0,12.666667,0.666667,4.333333,0,0.0,3.666667,1.39,coumarin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,10.1016/j.dyepig.2012.02.014,595.74,8.891642,0.730752,6,0.140845,2,1,28,3,...,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0.000000,0.41,triphenylamine
1459,10.1016/j.dyepig.2012.02.014,563.74,8.672923,0.739252,6,0.115942,2,1,28,3,...,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0.000000,0.44,triphenylamine
1460,10.1021/am403668d,665.84,7.481348,0.685335,5,0.182796,4,1,29,3,...,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0.000000,5.94,triphenylamine
1461,10.1021/am403668d,707.93,7.223776,0.676316,7,0.176471,4,1,29,3,...,0.0,0.0,0.000000,0.000000,0.000000,0,0.0,0.000000,5.88,triphenylamine


In [12]:
from sklearn.model_selection import train_test_split

modelSet, evalSet = train_test_split(dragonDatset, test_size=0.2, random_state=0)

evalSetCols = list(evalSet.columns)

evalSet_X = evalSet[evalSetCols[1:799]] # X values for all
evalSet_Y = evalSet['PCE']

modelSet.shape, evalSet.shape


((1170, 801), (293, 801))

Family Datasets

In [13]:
def splitDataset(globalDf, familyList):
    familyDfs = []
    for i in range(len(familyList)):
        familyDfs.append(globalDf.loc[(globalDf['Molecule keywords'].str.contains(familyList[i]))])
    familyDfs.append(
        globalDf.loc[
            (~globalDf['Molecule keywords'].str.contains('triphenylamine')) 
                            & (~globalDf['Molecule keywords'].str.contains('phenothiazine')) 
                            & (~globalDf['Molecule keywords'].str.contains('carbazole')) 
                            & (~globalDf['Molecule keywords'].str.contains('indoline')) 
                            & (~globalDf['Molecule keywords'].str.contains('coumarin')) 
                            & (~globalDf['Molecule keywords'].str.contains('diphenylamine'))
        ]
    )
    return familyDfs

Get Specific Descriptors

In [14]:
def getDragonDescriptors(dataframe):
    cols = list(dataframe.columns)
    X = dataframe[cols[1:799]]
    return X

Scaling Data

In [15]:
from sklearn.preprocessing import StandardScaler

def scaleData(X_data):
    scaled_array = StandardScaler().fit_transform(X_data)
    scaledDataFrame = pd.DataFrame(scaled_array, index=X_data.index, columns=X_data.columns)
    return scaledDataFrame

Reduce the number of features for each family

In [16]:
cmpdFamList= ['triphenylamine', 'phenothiazine', 'carbazole', 'indoline', 'coumarin', 'diphenylamine']
cmpdFamList
familyDfs = splitDataset(modelSet, cmpdFamList)

In [26]:
glob_train_X, glob_test_X, glob_train_Y, glob_test_Y = train_test_split(scaleData(getDragonDescriptors(modelSet)), modelSet['PCE'], test_size=0.2, random_state=0)
triph_train_X, triph_test_X, triph_train_Y, triph_test_Y = train_test_split(scaleData(getDragonDescriptors(familyDfs[0])), familyDfs[0]['PCE'], test_size=0.2, random_state=0)
phen_train_X, phen_test_X, phen_train_Y, phen_test_Y = train_test_split(scaleData(getDragonDescriptors(familyDfs[1])), familyDfs[1]['PCE'], test_size=0.2, random_state=0)
carb_train_X, carb_test_X, carb_train_Y, carb_test_Y = train_test_split(scaleData(getDragonDescriptors(familyDfs[2])), familyDfs[2]['PCE'], test_size=0.2, random_state=0)
indo_train_X, indo_test_X, indo_train_Y, indo_test_Y = train_test_split(scaleData(getDragonDescriptors(familyDfs[3])), familyDfs[3]['PCE'], test_size=0.2, random_state=0)
coum_train_X, coum_test_X, coum_train_Y, coum_test_Y = train_test_split(scaleData(getDragonDescriptors(familyDfs[4])), familyDfs[4]['PCE'], test_size=0.2, random_state=0)
diph_train_X, diph_test_X, diph_train_Y, diph_test_Y = train_test_split(scaleData(getDragonDescriptors(familyDfs[5])), familyDfs[5]['PCE'], test_size=0.2, random_state=0)
othr_train_X, othr_test_X, othr_train_Y, othr_test_Y = train_test_split(scaleData(getDragonDescriptors(familyDfs[6])), familyDfs[6]['PCE'], test_size=0.2, random_state=0)

glob_train_X.shape, phen_train_X.shape, carb_train_X.shape, indo_train_X.shape, coum_train_X.shape, diph_train_X.shape, othr_train_X.shape

((936, 798),
 (176, 798),
 (124, 798),
 (66, 798),
 (32, 798),
 (18, 798),
 (140, 798))

Feature Selection

RFE With SVR and RFR

In [18]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt

svr = SVR(kernel="linear")
rfr = RandomForestRegressor()
# svr.fit(triph_train_X, triph_train_Y)

min_features = 1
rfecv = RFECV(estimator=rfr, step=1, cv=2, scoring='r2', min_features_to_select=min_features)

rfecv.fit(othr_train_X, othr_train_Y)

KeyboardInterrupt: 

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)
# print(rfecv.cv_results_)

# RF triph 517
# phen 542 (but could use just 100)
# carb 582 (but again it plateaus at 100)
# indo 656 (let's try 60 features)
# coum 713 (again the distribution is completely square)
# diph 560 (This time it really did go up with the number of features but we're talking about only 18 instances here)
# othr 677 (but again, square data is fine)

Optimal number of features : 677


In [ ]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (r2)")
plt.plot(
    range(
        0, len(rfecv.feature_names_in_)
    ), rfecv.cv_results_['mean_test_score']

)
plt.savefig('featureSelectionRFothr')

Feature Selection

Disregarding Y

Variance Threshold

In [19]:
# will be done by length of the datasets themselves
# bottom up approach

from sklearn.feature_selection import VarianceThreshold

def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

# test until square data

Using Y

Univariate

In [27]:
from sklearn.feature_selection import SelectKBest, f_regression, SelectPercentile

def select_univariate(input_data_X, input_data_Y):
    if len(input_data_X) > len(input_data_X.columns):
        X = SelectPercentile(score_func=f_regression, percentile=30)
    else:
        X = SelectKBest(f_regression, k=len(input_data_X)).fit_transform(input_data_X, input_data_Y)
    return X

Sequential Feature Selection Ridge

In [31]:
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel
from sklearn.linear_model import RidgeCV, Ridge, LassoCV, Lasso

def sequen_features(input_X, input_Y):
    ridge = RidgeCV(alphas=np.logspace(-6,6,num=5)).fit(input_X, input_Y)
    if len(input_X) > len(input_X.columns):
        X=SequentialFeatureSelector(ridge, n_features_to_select=0.3, direction="forward").fit_transform(input_X, input_Y)
    else:
        X=SequentialFeatureSelector(ridge, n_features_to_select=len(input_X), direction="forward").fit_transform(input_X,input_Y)
    return X



Select from Model - Lasso

In [23]:
#this will give features that are linearly aligned

def lasso_features(input_X, input_Y):
    lasso = LassoCV(alphas=np.arange(0.1, 10, 0.1)).fit(input_X, input_Y)
    model = SelectFromModel(lasso, prefit=True)
    X_new = model.transform(input_X)
    return X_new
    

Select from Model - Tree

In [24]:
from sklearn.ensemble import ExtraTreesRegressor


def tree_features(input_X, input_Y):
    treeReg = ExtraTreesRegressor(n_estimators=100)
    treeReg = treeReg(input_X, input_Y)
    model = SelectFromModel(treeReg, prefit=True)
    X_new = model.transform(input_X)
    return X_new

Store Methods of Feature Selection

In [32]:
currentFamilySets = [   [glob_train_X, glob_train_Y],
                        [triph_train_X,triph_train_Y],
                        [phen_train_X, phen_train_Y],
                        [carb_train_X, carb_train_Y],
                        [indo_train_X, indo_train_Y],
                        [coum_train_X, coum_train_Y],
                        [diph_train_X, diph_train_Y],
                        [othr_train_X, othr_train_Y]
                    ]

def allMethodsFeatureSelect(familyTrainingData):
    selected_X = []
    for i in range(len(familyTrainingData)):
        selected_X.append(
                            [remove_low_variance(familyTrainingData[i][0], threshold=0.5),
                            select_univariate(familyTrainingData[i][0],familyTrainingData[i][1]),
                            sequen_features(familyTrainingData[i][0],familyTrainingData[i][1]),
                            lasso_features(familyTrainingData[i][0],familyTrainingData[i][1]),
                            tree_features(familyTrainingData[i][0],familyTrainingData[i][1])
                            ]
                        
                        )
    return selected_X

selected_X = allMethodsFeatureSelect(currentFamilySets)

selected_X[0][0].shape

Models

Ridge

In [ ]:
from sklearn.model_selection import GridSearchCV
# GridSearchCV for best parameters in training set
tuned_params_Ridge = {'alpha' : np.arange(0.1, 10, 0.1), 'fit_intercept' : [True, False]}

gsRidge = GridSearchCV(Ridge(), tuned_params_Ridge, cv=3, scoring='r2')
gsRidge.fit(selected_X[0][0], currentFamilySets[0][1])

# Apply model with best obtained parameters

# Validate with family test set

# Grid of results Y Pred vs Y True
#   For each family --> results_X[i]
#       For each feature select method --> results_X[i][j]

Lasso

PLS

Find Latent Variables

SVR - Linear Kernel

SVR - RBF Kernel

Random Forest

GTM

In [ ]:
from ugtm import eGTR
from sklearn.model_selection import GridSearchCV

tuned_params = {'regul': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                's': [0.1, 0.2, 0.3],
                'k': [0, 16, 25],
                'm': [4, 5]
}

gs = GridSearchCV(eGTR(), tuned_params, cv=3, scoring='r2') # error_score='raise'
gs.fit(triph_train_X, triph_train_Y)
print(gs.best_params_)

C:\Users\Paul\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
126 fits failed out of a total of 378.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
126 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Paul\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Paul\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ugtm\ugtm_sklearn.py", line 456, in fit
    self.initialModel = ugtm_gtm.initialize(X, self.k,
  File "C:\Users\Paul\AppData\Local\Programs\Python\Python38-32\lib\

{'k': 16, 'm': 5, 'regul': 0.001, 's': 0.3}


In [ ]:
print(gs.best_score_)

0.20175921560726226


In [ ]:
gtr = eGTR(k=16, m=4, regul=0.0001, s=0.3) # 

gtr = gtr.fit(othr_train_X, othr_train_Y)



Visualise GTR

In [ ]:
import altair as alt

dfclassmap = pd.DataFrame(gtr.optimizedModel.matX, columns=["x1","x2"])
dfclassmap["predicted_node_label"] = gtr.node_label

alt.Chart(dfclassmap).mark_square().encode(
    x='x1',
    y='x2',
    color=alt.Color(
        'predicted_node_label:Q',
        scale = alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title="PCE")
    ),
    size=alt.value(50),
    tooltip=['x1','x2', 'predicted_node_label:Q']
).properties(title="Dye-Sensitised Solar Cells", width=200, height=200)

alt.Chart(...)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

Y_pred_train = gtr.predict(othr_train_X)
Y_pred_test = gtr.predict(othr_test_X)

mae_train = mean_absolute_error(othr_train_Y, Y_pred_train)
mae_test = mean_absolute_error(othr_test_Y, Y_pred_test)

r2_train = r2_score(othr_train_Y, Y_pred_train)
r2_test = r2_score(othr_test_Y, Y_pred_test)

mae_train, mae_test, r2_train, r2_test

(0.5932559500104309,
 1.3763942858013114,
 0.8199555459804353,
 0.012036851053051523)